### package 불러오기 (환경 설정)

In [3]:
import torch 
from torch.utils.data import DataLoader

import numpy as np

from include.model import ConvNet
from include.dataset import RadarDataset 
from include.utils import preprocessing_rx_sig, postprocessing

c:\Users\user\anaconda3\envs\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### 변수 설정

In [4]:
# 모델 파라미터
BATCH_SIZE = 50
BATCH_MOMENTM = 0.9
EPOCH = 1
lr = 5e-6   # 5e-6 
device = torch.device('cuda' if torch.cuda.is_available() else "cpu")

# 레이더 각도 변수
MIN_ANGLE, MAX_ANGLE, RESOLUTION = -60, 60, 10
NUM_CLASS = (MAX_ANGLE-MIN_ANGLE) // RESOLUTION + 1

print(NUM_CLASS)

13


### 학습된 모델 불러오기

In [8]:
model_checkpoint = './model/model_10_60_12232144.ckpt'

model = ConvNet(NUM_CLASS=NUM_CLASS, BATCH_MOMENTM=BATCH_MOMENTM).to(device)
model.load_state_dict(torch.load(model_checkpoint))

<All keys matched successfully>

### 실험 데이터 불러오기

In [9]:
FIRST_IDX = 1
PATH_LIST = ['./data/23_12_22_data/data', './data/23_12_16_data/data']
SIMULATION_INCLUDE = False

# experiment data
for idx, path_val in enumerate(PATH_LIST, 1):
    if idx == FIRST_IDX:
        Rx_sig, angle = preprocessing_rx_sig(path=path_val, min_angle=MIN_ANGLE, 
                                             max_angle=MAX_ANGLE, resolution=RESOLUTION)
    else:
        Rx_sig_tmp, angle_tmp = preprocessing_rx_sig(path=path_val, min_angle=MIN_ANGLE, 
                                                     max_angle=MAX_ANGLE, resolution=RESOLUTION)
        Rx_sig = np.concatenate((Rx_sig, Rx_sig_tmp))
        angle = np.concatenate((angle, angle_tmp))

# Simulation data
if SIMULATION_INCLUDE:
    Rx_sig_simul = np.load(f'./data/simul/total_output_COV_{RESOLUTION}_{MAX_ANGLE}.npy')
    angle_simul  = np.load(f'./data/simul/total_output_angle_{RESOLUTION}_{MAX_ANGLE}.npy')
    angle_simul = np.argmax(angle_simul, axis=1)

    Rx_sig_simul /= np.max(Rx_sig_simul)    # normalization

    # concatenate experiment with simulation
    Rx_sig = np.concatenate((Rx_sig_simul, Rx_sig))
    angle = np.concatenate((angle_simul, angle))

# convert input shape - [real, imag, phase]
x_test = np.array([[(np.real(Rxx)), (np.imag(Rxx)), (np.angle(Rxx))] for Rxx in Rx_sig])

# print size
print('x_test :', len(x_test))

# dataloader로 변환
test_set = RadarDataset(transform=torch.from_numpy, data=x_test, label=angle)
test_loader = DataLoader(dataset=test_set, batch_size=BATCH_SIZE, shuffle = True)

x_test : 12132


### 결과 확인 

In [10]:
angle_range = torch.arange(MIN_ANGLE, MAX_ANGLE+1, RESOLUTION)
TOLERANCE_ANGLE = 10
ZERO_INDEX = NUM_CLASS // 2

model.eval()
with torch.no_grad():
    sum_correct = 0
    
    for batch_idx, (x_test, y_test) in enumerate(test_loader, 1):
        x_test = x_test.to(device)
        real = y_test
        outputs = model(x_test)
    
        # 조건 설정
        outputs[outputs < 0.1] = 0                                         # 1. 0.1보다 작으면 죽이기
        outputs[:, ZERO_INDEX][outputs[:, ZERO_INDEX] < 0.5] = 0           # 2. 0이 0.5보다 작으면 죽이기
        pred = postprocessing(outputs, NUM_CLASS, RESOLUTION, MIN_ANGLE)   # 3. top1보다 그 이외의 값의 합이 더 크면 수정
        
        # 성능 평가
        correct = np.sum(list(map(lambda x, y : np.abs(x-y) <= TOLERANCE_ANGLE, real, pred)))
        sum_correct += correct
        
    print(f'{round(sum_correct / (batch_idx * BATCH_SIZE), 2) * 100}%')

89.0%
